In [13]:
"""
Created on Tue March 8th, 2022
@author: Eleftheria Chatzitheodoridou

Description: 
This script is designed to read NIfTI files that contain Grade 4 (HGG) tumors from the
local directory, extract the areas where the tumor is present across (x,y,z) by finding
the (min, max) of each axis in the annotations directory, then normalize the intensity of 
the extracted 2D images and save the slices on a local directory as .png files.
"""

import os.path 
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from PIL import Image
from numpy import ndarray

# Folder where the created images will be saved in
out_path = r'/local/data1/elech646/Tumor_grade_classification/Slices'
dataset_path = r'/local/data1/elech646/Tumor_grade_classification/HGG'

# Create subfolders
if not os.path.exists(out_path + "/sagittal_grade_classification"):
    os.mkdir(out_path + "/sagittal_grade_classification")
    
if not os.path.exists(out_path + "/frontal_grade_classification"):
    os.mkdir(out_path + "/frontal_grade_classification")

if not os.path.exists(out_path + "/trans_grade_classification"):
    os.mkdir(out_path + "/trans_grade_classification")

# Add HGG path
sag_path = out_path + "/sagittal_grade_classification" + "/HGG"
fro_path = out_path + "/frontal_grade_classification" + "/HGG"
tra_path = out_path + "/trans_grade_classification" + "/HGG"
    
if not os.path.exists(sag_path):
    os.mkdir(sag_path)
    
if not os.path.exists(fro_path):
    os.mkdir(fro_path)
    
if not os.path.exists(tra_path):
    os.mkdir(tra_path)

In [14]:
seg_path = []
patient_name = []

for roots, dirs, files in os.walk("/local/data1/elech646/Tumor_grade_classification/HGG_tumor_annotations"):
    for name in files:
        if name.endswith((".nii.gz",".nii")):
            seg_path.append(roots + os.path.sep + name)
            patient_name.append('_'.join(name.split('_')[:3]))

# idx = [i for i, v in enumerate(patient_name) if 'Training_001' in v][0]
# patient_name = [patient_name[idx]]
# seg_path = [seg_path[idx]]

# Loop through the subjects
for p_name, s_path in zip(patient_name, seg_path):
    # Load the segmentation file
    seg_img = nib.load(s_path)
    seg_img_data = seg_img.get_fdata()
    
    # Create subfolders 
    if not os.path.exists(os.path.join(sag_path, p_name)):
        os.mkdir(os.path.join(sag_path,p_name))
    if not os.path.exists(os.path.join(fro_path, p_name)):
        os.mkdir(os.path.join(fro_path,p_name))
    if not os.path.exists(os.path.join(tra_path, p_name)):
        os.mkdir(os.path.join(tra_path,p_name))
    
    # Loop through the modalities
    modalities = ['t1', 't1ce', 't2', 'flair']
                 
    for m in modalities:
        #print(f'Working on {p_name}, modality {m} \r', end = '')
        # Load full image for this modality
        #mod_img = nib.load(os.path.join(dataset_path, patient_name[i], patient_name[i] + '_' + idx + '.nii'))
        mod_img = nib.load(os.path.join(dataset_path, p_name, '_'.join([p_name, m]) + '.nii'))
        mod_img_data = mod_img.get_fdata()
        
        # Creating the images in the Sagittal Plane (yz)      
        img_sag = np.rot90(mod_img_data, axes = (1, 2)) # yz plane sagittal
        img_sag = np.flip(img_sag, 0) # flip the image left/right
        
        # Do some weird voodoo magic with rotations because Mango
        seg_img_data_sag = np.rot90(seg_img_data, axes = (1, 2)) # yz plane sagittal
        seg_img_data_sag = np.flip(seg_img_data_sag, 0) # flip the image left/right 
                 
        # Get indices
        sag_0 = min(ndarray.nonzero(seg_img_data_sag)[0])   # zmin
        sag_1 = max(ndarray.nonzero(seg_img_data_sag)[0])   # zmax
        
        # Checking indices for sagittal case
#         fig, ax = plt.subplots(nrows = 2, ncols = 2)
#         ax[0,0].imshow(img_sag[sag_0,:,:], cmap = 'gray', interpolation = None)
#         ax[0,0].set_title(f'Slice {sag_0}')
#         ax[0,1].imshow(img_sag[sag_1,:,:], cmap = 'gray', interpolation = None)
#         ax[0,1].set_title(f'Slice {sag_1}')
        
#         ax[1,0].imshow(seg_img_data_sag[sag_0,:,:], cmap = 'gray', interpolation = None)
#         ax[1,0].set_title(f'Slice {sag_0}')
#         ax[1,1].imshow(seg_img_data_sag[sag_1,:,:], cmap = 'gray', interpolation = None)
#         ax[1,1].set_title(f'Slice {sag_1}')       
#         plt.show()        
                 
        for sag in range(sag_0, sag_1 + 1):
            perc = int(((sag - sag_0)/(sag_1 - sag_0))*100) # Percentage along the selected slices
            tmp = img_sag[sag,:,:]
            min_v = img_sag.min()
            max_v = img_sag.max()

            # Normalize image
            tmp_norm = (255*(tmp - min_v) / (max_v - min_v)).astype(np.uint8)

            # Name the files
            title = os.path.join(sag_path, p_name, p_name + '_sag_' + m +\
                                 '_' + str(sag) + '_' + str(perc) + '.png')
            # Convert to RGB
            im = Image.fromarray(tmp_norm).convert('RGB')

            # Save images
            im.save(title)            
            #print(title) 
        
        # Creating the images in the Frontal/Coronal Plane (xz)
        img_fr = np.rot90(mod_img_data, axes = (0,2)) # xz plane frontal 
        
        # Do some weird voodoo magic with rotations because Mango
        seg_img_data_fr = np.rot90(seg_img_data, axes = (0,2)) # yz plane sagittal
        
        # Get indices
        fr_0 = min(ndarray.nonzero(seg_img_data_fr)[2])    # ymin, both 1, 2 work
        fr_1 = max(ndarray.nonzero(seg_img_data_fr)[2])    # ymax
        
        # Checking indices for frontal case
#         fig, ax = plt.subplots(nrows = 2, ncols = 2)
#         ax[0,0].imshow(img_fr[:,fr_0,:], cmap = 'gray', interpolation = None)
#         ax[0,0].set_title(f'Slice {fr_0}')
#         ax[0,1].imshow(img_fr[:,fr_1,:], cmap = 'gray', interpolation = None)
#         ax[0,1].set_title(f'Slice {fr_1}')

#         ax[1,0].imshow(seg_img_data_fr[:,fr_0,:], cmap = 'gray', interpolation = None)
#         ax[1,0].set_title(f'Slice {fr_0}')
#         ax[1,1].imshow(seg_img_data_fr[:,fr_1,:], cmap = 'gray', interpolation = None)
#         ax[1,1].set_title(f'Slice {fr_1}')
#         plt.show()  
        
        for front in range(fr_0, fr_1 + 1):
            perc = int(((front - fr_0) /(fr_1 - fr_0))*100) # Percentage along the selected slices
            tmp = img_fr[:,front,:]
            min_v = img_fr.min()
            max_v = img_fr.max()

            # Normalize image
            tmp_norm = (255*(tmp - min_v) / (max_v - min_v)).astype(np.uint8) 

            # Name the files
            title = os.path.join(fro_path, p_name, p_name + '_fro_' + m +\
                                 '_' + str(front) + '_' + str(perc) + '.png')
            # Convert to RGB
            im = Image.fromarray(tmp_norm).convert('RGB')

            # Save images
            im.save(title)         

        # Creating the images in the Transversal/Axial Plane (xy)
        img_tr = np.rot90(mod_img_data, 3, axes = (0,1)) # xy plane transversal
        #x,y,z = img_tr.shape

        # Do some weird voodoo magic with rotations because Mango
        seg_img_data_tr = np.rot90(seg_img_data, 3, axes = (0,1)) # yz plane sagittal
        x_seg, y_seg, z_seg = seg_img_data_tr.shape

        # Inverting slices upside/down since mango was used
        tr_0 = z_seg - max(ndarray.nonzero(seg_img_data_tr)[2]) #1,2 both work
        tr_1 = z_seg - min(ndarray.nonzero(seg_img_data_tr)[2]) #1,2 both work
        
        # Checking indices for transversal case
#         fig, ax = plt.subplots(nrows = 2, ncols = 2)
#         ax[0,0].imshow(img_tr[:,:,tr_0], cmap = 'gray', interpolation = None)
#         ax[0,0].set_title(f'Slice {tr_0}')
#         ax[0,1].imshow(img_tr[:,:,tr_1], cmap = 'gray', interpolation = None)
#         ax[0,1].set_title(f'Slice {tr_1}')

#         ax[1,0].imshow(seg_img_data_tr[:,:, tr_0], cmap = 'gray', interpolation = None)
#         ax[1,0].set_title(f'Slice {tr_0}')
#         ax[1,1].imshow(seg_img_data_tr[:,:, tr_1], cmap = 'gray', interpolation = None)
#         ax[1,1].set_title(f'Slice {tr_1}')
#         plt.show()

        for transv in range(tr_0, tr_1):
            perc = int(((transv - tr_0) /(tr_1 - tr_0))*100) # Percentage along the selected slices
            tmp = img_tr[:,:,transv]
            min_v = img_tr.min()
            max_v = img_tr.max()

            # Normalize image
            tmp_norm = (255*(tmp - min_v) / (max_v - min_v)).astype(np.uint8) 

            # Name the files
            title = os.path.join(tra_path, p_name, p_name + '_trans_' + m +\
                                 '_' + str(transv) + '_' + str(perc) + '.png')
            # Convert to RGB
            im = Image.fromarray(tmp_norm).convert('RGB')

            # Save images
            im.save(title)         

In [ ]:
# Checking where tumor slices are

# transversal case
#print(np.argwhere(seg_img_data_tr.sum(axis = (0,1)) > 0))  # 58-134

# frontal case
#print(np.argwhere(seg_img_data_fr.sum(axis = (0,1)) > 0))  # 136-183

# sagittal case
#print(np.argwhere(seg_img_data_sag.sum(axis = (0,1)) > 0)) # 93-186